<a href="https://colab.research.google.com/github/jolex007/COS_DL/blob/master/1_HomeWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Задание по машинке

In [ ]:
import numpy as np
import os
import cv2

## Инициализация модели

In [ ]:
def init_model(input_size=256):
    np.random.seed(2)
    w = np.random.randn(input_size ** 2, 1)
    b = 1.
    return w, b

In [ ]:
def sigmoid(x):
    return 1. / (1 + np.exp(-x))

## Forward & Backward propogation

In [ ]:
def propagate(w, b, X, Y):
    """
    Подсчет текущего предсказания (оно же forward propagation) и градиента функции ошибки (оно же backward propagation)

    Input:
    w -- веса, numpy_array размера (num_px * num_px * 3, 1)
    b -- смещение, скалярная величина
    X -- данные размера (num_px * num_px * 3, кол-во образцов)
    Y -- вектор истинных ответов размера (1, кол-во образцов)

    Return:
    cost -- текущая функция ошибки
    dw -- градиент функции ошибки по w
    db -- градиент функции ошибки по b (по сути производная по b)
    
    """
    
    m = X.shape[1]
    
    print("X: ", X)
    print("Some other trash: ", w.T @ X + b)

    A = sigmoid(w.T @ X + b)

    print("A: ", A)
    print("log(A): ", np.log(A))
    print("log(1 - A): ", np.log(1 - A))
    print("Y: ", Y)

    cost = (-1. / m) * (Y * np.log(A) + (1 - Y) * np.log(1 - A)).sum()
    print("Some trash: ", (Y * np.log(A) + (1 - Y) * np.log(1 - A)))
    
    dw = (1. / m) * X @ (A - Y).T
    db = (1. / m) * (A - Y).sum()
    

    grads = {"dw": dw,
             "db": db}
    
    print("!!!!grads :", grads)

    return grads, cost

## Шаг обновления весов

In [ ]:
def optimize(w, b, X, Y, num_iterations, learning_rate=0.06, print_cost=False):
    """
    Оптимизация с помощью простого градиентного спуска
    
    Input:
    w -- веса, numpy_array размера (num_px * num_px * 3, 1)
    b -- смещение, скалярная величина
    X -- данные размера (num_px * num_px * 3, кол-во образцов)
    Y -- вектор истинных ответов размера (1, кол-во образцов)
    num_iterations -- кол-во итераций алгоритма оптимизации
    learning_rate -- коэффициент learning rate
    print_cost -- True, если хотите выводить функцию ошибки на каждых 100 итерациях
    
    Returns:
    params -- словарь, содержащий w и b
    grads -- словарь, содержащий градиенты функции ошибки по w и b соответственно
    costs -- массив (list) со значением функции ошибки для каждой итерации (так делают для визуализации)
    
    Подсказка:
    
        1) Используйте ранее написанную функцию propagate().
        2) Обновляйте параметры w и b согласно формуле 10.
    """
    
    costs = []
    
    for i in range(num_iterations):
        
        
        ###Напишите значения для градиентов и функции ошибки
        grads, cost = propagate(w, b, X, Y)
        
        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # обновление параметров
        
        ### START CODE HERE ###
        w = w - learning_rate * dw
        b = b - learning_rate * db
        ### END CODE HERE ###
        
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training iterations
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

## Предсказание модели

In [ ]:
def predict(w, b, X):
    '''
    Inputs:
    w
    b
    X -- данные размера (num_px * num_px * 3, кол-во образцов)
    
    Returns:
    Y_prediction
    '''
    
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    ### START CODE HERE ### (≈ 1 line of code)
    A = sigmoid(w.T @ X + b)
    ### END CODE HERE ###
    
    for i in range(A.shape[1]):
        # Установите порог, выше которого считаем, что модель выдает 1, а ниже - ноль
        ### START CODE HERE ###
        if A[0][i] <= 0.7:
            Y_prediction[0][i] = 0
        else:
            Y_prediction[0][i] = 1
        ### END CODE HERE ###
    
    
    return Y_prediction

In [ ]:
def model(X, Y, num_iterations=10000, print_cost=False):
    w, b = init_model()
    optimize(w, b, X, Y, num_iterations, print_cost=print_cost)
    
    return w, b

## Обработка датасета

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
def read_files(X, Y, path, ans):
    files = os.listdir(path)
    for name in files:
        img = cv2.imread(path + '/' + name, 0)
        if img.shape != 0:
            img = cv2.resize(img, (256, 256))
            vect = img.reshape(1, 256 ** 2)
            vect = vect / 255.
            X = vect if (X is None) else np.vstack((X, vect)) 
            Y = np.append(Y, ans)
    return X, Y

path = "gdrive/My Drive/COS/lesson1_dataset"
# os.listdir("content/gdrive/")
X = None
Y = np.array([])
X, Y = read_files(X, Y, path + "/logloss_0", 0)
X, Y = read_files(X, Y, path + "/logloss_1", 1)

X = X.T

## Запуск модели

In [ ]:
w, b = model(X, Y, num_iterations=10, print_cost=True)

X:  [[0.02745098 0.01960784 0.01176471 ... 0.10196078 0.09803922 0.11372549]
 [0.03137255 0.02352941 0.01568627 ... 0.10980392 0.10980392 0.13333333]
 [0.03921569 0.02745098 0.01960784 ... 0.1254902  0.12156863 0.14901961]
 ...
 [0.07843137 0.05882353 0.03529412 ... 0.04313725 0.04705882 0.05882353]
 [0.0745098  0.05490196 0.03529412 ... 0.04313725 0.03921569 0.05490196]
 [0.07058824 0.05098039 0.03137255 ... 0.03921569 0.03921569 0.05098039]]
Some other trash:  [[-49.24552937 -36.80153519 -24.78007453 -12.42950465 -62.94378615
  -77.1033811  -92.4270803  -69.65063561  -8.72687991  -9.253647
  -99.21946975 -30.7280788  -85.47532543 -81.10269974 -60.40470149
  -52.20172442 -14.84001083 -21.99466449 -34.28915713 -33.48572447
  -18.67771882 -62.40153015  63.73571187 -23.1597478    6.55148964
   31.3650115   12.31661098 -19.10834117 -12.03830045 -41.36433933
   22.05169325   8.57722588 -28.9758625  -16.59961022  26.61404277
  -72.6847077  -23.9340392  -12.36307791 -41.03505368   1.11943327

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in multiply



log(1 - A):  [[-7.36832817e-11 -2.66079614e-08 -6.50101161e-06 -2.93642713e-03
  -3.85247390e-14  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -7.52223394e+00 -8.04867298e+00  0.00000000e+00 -2.97677554e-06
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.11022302e-16
  -5.27224925e-01 -5.36969456e-04 -1.72034231e-08 -6.87814860e-08
  -1.41648492e+01  0.00000000e+00            -inf -7.31353773e-01
             -inf            -inf            -inf -2.34430182e+01
  -1.03866076e+01 -1.84487902e-08            -inf -2.32439142e+01
  -4.98857995e-01 -5.62464428e+00            -inf -5.42899059e-13
  -4.52417892e-01 -3.13251545e+01 -7.55766041e-09 -1.54826988e+01
             -inf -1.41409436e-04            -inf -9.69162067e+00
             -inf -1.10293528e-04 -1.09817575e+01            -inf
             -inf -1.60641657e+01            -inf -1.73251732e-08
  -3.69930753e-11 -8.14367169e-08            -inf -4.36195252e-02
  -3.35587467e+01 -2.51574658e-06 -3.34045961e+01 -8.57871892e

In [ ]:
result = predict(w, b, X)

In [ ]:
print(result)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1.
  1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1.]]
